# Location search for restaurant in the Netherlands

### Import relevant tools

In [1]:
# Import tools
import requests
import pandas as pd
import numpy as np
!conda install -c conda-forge geopy --yes 
import folium # plotting library

import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

print('tools imported')

Solving environment: ...working... done

# All requested packages already installed.

tools imported


### Import Datasets

#### Import Dutch cities with geo information

In [2]:
geo_data = pd.read_csv("NL_geo.csv")
geo_data.head(10)

,city,lat,lng,country,iso2,admin,capital,population,population_proper
0,Den Haag,52.083333,4.300000,Netherlands,NL,Zuid-Holland,primary,1406000.0,501725.0
1,Amsterdam,52.350000,4.916667,Netherlands,NL,Noord-Holland,primary,1031000.0,741636.0
2,Rotterdam,51.916667,4.500000,Netherlands,NL,Zuid-Holland,minor,1005000.0,598199.0
3,Utrecht,52.093813,5.119095,Netherlands,NL,Utrecht,admin,640000.0,316448.0
4,Eindhoven,51.450000,5.466667,Netherlands,NL,Noord-Brabant,minor,398053.0,209620.0
5,Haarlemmerliede,52.388873,4.687978,Netherlands,NL,Noord-Holland,NaN,349957.0,147590.0
6,Groningen,53.216667,6.550000,Netherlands,NL,Groningen,admin,216688.0,181194.0
7,Arnhem,51.979818,5.925636,Netherlands,NL,Gelderland,admin,141674.0,141674.0
8,Den Bosch,51.700196,5.306806,Netherlands,NL,Noord-Brabant,admin,134520.0,134520.0
9,Leeuwarden,53.201390,5.808588,Netherlands,NL,Fryslân,admin,125778.0,91424.0


In [3]:
geo_data.shape

(408, 9)

#### Import Dutch city information including Boroughs and Neigbourhoods

In [4]:
# Dataset is avaialable from CBS, the Dutch statistics organisation as excel file
city_data = pd.read_excel('CBS Buurten kwb-2019.xls')
city_data.head()

,Unnamed: 0,gwb_code_8,regio,gm_naam,recs,gwb_code,ind_wbi,a_inw,a_man,a_vrouw,...,g_afs_kv,g_afs_sc,g_3km_sc,a_opp_ha,a_lan_ha,a_wat_ha,pst_mvp,pst_dekp,ste_mvs,ste_oad
0,NL00,0,Nederland,Nederland,Land,NL00,.,17282163,8581086,8701077,...,"0,6",NaN,NaN,4154338,3367109,787228,.,.,2,1994
1,GM0003,3,Appingedam,Appingedam,Gemeente,GM0003,.,11721,5720,6001,...,"0,9",NaN,NaN,2458,2378,80,.,.,3,1067
2,WK000300,300,Wijk 00,Appingedam,Wijk,WK000300,1,11720,5720,6000,...,"0,9",NaN,NaN,2458,2378,80,.,.,3,1067
3,BU00030000,30000,Appingedam-Centrum,Appingedam,Buurt,BU00030000,1,2370,1135,1230,...,"1,2",NaN,NaN,90,84,5,9901,1,3,1210
4,BU00030001,30001,Appingedam-West,Appingedam,Buurt,BU00030001,1,3035,1505,1530,...,"1,0",NaN,NaN,163,158,5,9903,6,4,903


In [5]:
city_data.shape

(17114, 109)

#### Rename columns

In [6]:
city_data.rename(columns={'gm_naam': 'city'}, inplace = True)
city_data.head()

,Unnamed: 0,gwb_code_8,regio,city,recs,gwb_code,ind_wbi,a_inw,a_man,a_vrouw,...,g_afs_kv,g_afs_sc,g_3km_sc,a_opp_ha,a_lan_ha,a_wat_ha,pst_mvp,pst_dekp,ste_mvs,ste_oad
0,NL00,0,Nederland,Nederland,Land,NL00,.,17282163,8581086,8701077,...,"0,6",NaN,NaN,4154338,3367109,787228,.,.,2,1994
1,GM0003,3,Appingedam,Appingedam,Gemeente,GM0003,.,11721,5720,6001,...,"0,9",NaN,NaN,2458,2378,80,.,.,3,1067
2,WK000300,300,Wijk 00,Appingedam,Wijk,WK000300,1,11720,5720,6000,...,"0,9",NaN,NaN,2458,2378,80,.,.,3,1067
3,BU00030000,30000,Appingedam-Centrum,Appingedam,Buurt,BU00030000,1,2370,1135,1230,...,"1,2",NaN,NaN,90,84,5,9901,1,3,1210
4,BU00030001,30001,Appingedam-West,Appingedam,Buurt,BU00030001,1,3035,1505,1530,...,"1,0",NaN,NaN,163,158,5,9903,6,4,903


### Filter dataset, only city data is needed 
#### In dutch city =  "Gemeente"

In [7]:
# dataseit contains different types of roes, based on thge columns "recs"
# recs = "Land": data for the total of the Netherlands
# recs = "Gemeente": data for the total of a city
# recs = "Wijk": data for the total of a Borough
# recs = "Buurt": data for the total of a Neighbourhood

# For this prohect only city data is needed

In [8]:
# Make dataframe with neighbourhoud information only
df_city = city_data[city_data.recs == 'Gemeente']
df_city.reset_index(inplace=True)
df_city.head()

,index,Unnamed: 0,gwb_code_8,regio,city,recs,gwb_code,ind_wbi,a_inw,a_man,...,g_afs_kv,g_afs_sc,g_3km_sc,a_opp_ha,a_lan_ha,a_wat_ha,pst_mvp,pst_dekp,ste_mvs,ste_oad
0,1,GM0003,3,Appingedam,Appingedam,Gemeente,GM0003,.,11721,5720,...,"0,9",NaN,NaN,2458,2378,80,.,.,3,1067
1,9,GM0010,10,Delfzijl,Delfzijl,Gemeente,GM0010,.,24716,12408,...,"0,9",NaN,NaN,22750,13307,9443,.,.,4,673
2,67,GM0014,14,Groningen,Groningen,Gemeente,GM0014,.,231299,114937,...,"0,5",NaN,NaN,19796,18560,1236,.,.,1,3219
3,226,GM0024,24,Loppersum,Loppersum,Gemeente,GM0024,.,9614,4881,...,"1,4",NaN,NaN,11199,11104,95,.,.,5,213
4,254,GM0034,34,Almere,Almere,Gemeente,GM0034,.,207904,103082,...,"0,5",NaN,NaN,24877,12919,11958,.,.,2,1601


In [9]:
#### Show number of roew and columns of cities
df_city.shape

(355, 110)

#### Sort on number of inhabitants (in dutch the column: "a_inw")

In [10]:
df_sort = df_city.sort_values(by='a_inw', ascending=False)
df_sort.head()

,index,Unnamed: 0,gwb_code_8,regio,city,recs,gwb_code,ind_wbi,a_inw,a_man,...,g_afs_kv,g_afs_sc,g_3km_sc,a_opp_ha,a_lan_ha,a_wat_ha,pst_mvp,pst_dekp,ste_mvs,ste_oad
109,5006,GM0363,363,Amsterdam,Amsterdam,Gemeente,GM0363,.,862965,427787,...,"0,3",NaN,NaN,21949,16550,5399,.,.,1,6057
173,8354,GM0599,599,Rotterdam,Rotterdam,Gemeente,GM0599,.,644618,317935,...,"0,4",NaN,NaN,32416,21757,10659,.,.,1,3945
155,7760,GM0518,518,Den Haag,Den Haag,Gemeente,GM0518,.,537833,266778,...,"0,3",NaN,NaN,9813,8245,1568,.,.,1,4892
99,4533,GM0344,344,Utrecht,Utrecht,Gemeente,GM0344,.,352866,172857,...,"0,4",NaN,NaN,9921,9383,538,.,.,1,3394
212,9832,GM0772,772,Eindhoven,Eindhoven,Gemeente,GM0772,.,231642,119157,...,"0,5",NaN,NaN,8892,8766,126,.,.,1,2654


## Combine city data with geo data

In [11]:
df_merge = pd.merge(df_sort, geo_data,on = 'city')
df_merge.head()

,index,Unnamed: 0,gwb_code_8,regio,city,recs,gwb_code,ind_wbi,a_inw,a_man,...,ste_mvs,ste_oad,lat,lng,country,iso2,admin,capital,population,population_proper
0,5006,GM0363,363,Amsterdam,Amsterdam,Gemeente,GM0363,.,862965,427787,...,1,6057,52.350000,4.916667,Netherlands,NL,Noord-Holland,primary,1031000.0,741636.0
1,8354,GM0599,599,Rotterdam,Rotterdam,Gemeente,GM0599,.,644618,317935,...,1,3945,51.916667,4.500000,Netherlands,NL,Zuid-Holland,minor,1005000.0,598199.0
2,7760,GM0518,518,Den Haag,Den Haag,Gemeente,GM0518,.,537833,266778,...,1,4892,52.083333,4.300000,Netherlands,NL,Zuid-Holland,primary,1406000.0,501725.0
3,4533,GM0344,344,Utrecht,Utrecht,Gemeente,GM0344,.,352866,172857,...,1,3394,52.093813,5.119095,Netherlands,NL,Utrecht,admin,640000.0,316448.0
4,9832,GM0772,772,Eindhoven,Eindhoven,Gemeente,GM0772,.,231642,119157,...,1,2654,51.450000,5.466667,Netherlands,NL,Noord-Brabant,minor,398053.0,209620.0


In [12]:
df_merge.shape

(228, 118)

In [ ]:
# only for 217 out of 355 the merge is possible

In [13]:
df_merge

,index,Unnamed: 0,gwb_code_8,regio,city,recs,gwb_code,ind_wbi,a_inw,a_man,...,ste_mvs,ste_oad,lat,lng,country,iso2,admin,capital,population,population_proper
0,5006,GM0363,363,Amsterdam,Amsterdam,Gemeente,GM0363,.,862965,427787,...,1,6057,52.350000,4.916667,Netherlands,NL,Noord-Holland,primary,1031000.0,741636.0
1,8354,GM0599,599,Rotterdam,Rotterdam,Gemeente,GM0599,.,644618,317935,...,1,3945,51.916667,4.500000,Netherlands,NL,Zuid-Holland,minor,1005000.0,598199.0
2,7760,GM0518,518,Den Haag,Den Haag,Gemeente,GM0518,.,537833,266778,...,1,4892,52.083333,4.300000,Netherlands,NL,Zuid-Holland,primary,1406000.0,501725.0
3,4533,GM0344,344,Utrecht,Utrecht,Gemeente,GM0344,.,352866,172857,...,1,3394,52.093813,5.119095,Netherlands,NL,Utrecht,admin,640000.0,316448.0
4,9832,GM0772,772,Eindhoven,Eindhoven,Gemeente,GM0772,.,231642,119157,...,1,2654,51.450000,5.466667,Netherlands,NL,Noord-Brabant,minor,398053.0,209620.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223,8857,GM0638,638,Zoeterwoude,Zoeterwoude,Gemeente,GM0638,.,8450,4203,...,4,746,52.121128,4.496406,Netherlands,NL,Zuid-Holland,minor,NaN,NaN
224,9533,GM0744,744,Baarle-Nassau,Baarle-Nassau,Gemeente,GM0744,.,6847,3477,...,5,336,51.445401,4.929484,Netherlands,NL,Noord-Brabant,minor,NaN,NaN
225,4446,GM0339,339,Renswoude,Renswoude,Gemeente,GM0339,.,5259,2625,...,5,480,52.074493,5.540471,Netherlands,NL,Utrecht,minor,NaN,NaN
226,3616,GM0277,277,Rozendaal,Rozendaal,Gemeente,GM0277,.,1654,815,...,4,935,52.007364,5.965358,Netherlands,NL,Gelderland,minor,NaN,NaN


#### Size dataframe with top 50 cities only

In [14]:
subset = df_merge.iloc[0:50]
subset.head()

,index,Unnamed: 0,gwb_code_8,regio,city,recs,gwb_code,ind_wbi,a_inw,a_man,...,ste_mvs,ste_oad,lat,lng,country,iso2,admin,capital,population,population_proper
0,5006,GM0363,363,Amsterdam,Amsterdam,Gemeente,GM0363,.,862965,427787,...,1,6057,52.350000,4.916667,Netherlands,NL,Noord-Holland,primary,1031000.0,741636.0
1,8354,GM0599,599,Rotterdam,Rotterdam,Gemeente,GM0599,.,644618,317935,...,1,3945,51.916667,4.500000,Netherlands,NL,Zuid-Holland,minor,1005000.0,598199.0
2,7760,GM0518,518,Den Haag,Den Haag,Gemeente,GM0518,.,537833,266778,...,1,4892,52.083333,4.300000,Netherlands,NL,Zuid-Holland,primary,1406000.0,501725.0
3,4533,GM0344,344,Utrecht,Utrecht,Gemeente,GM0344,.,352866,172857,...,1,3394,52.093813,5.119095,Netherlands,NL,Utrecht,admin,640000.0,316448.0
4,9832,GM0772,772,Eindhoven,Eindhoven,Gemeente,GM0772,.,231642,119157,...,1,2654,51.450000,5.466667,Netherlands,NL,Noord-Brabant,minor,398053.0,209620.0


#### Size dataframe with relevant columny only

In [15]:
# Only relevant columns are selected.
# Because we are going to cluster the cities based on Foursquare information, figures of the different cities are niot relevant for now

df = subset[['city', 'a_inw', 'lat', 'lng']]
df.rename(columns={'a_inw':'number_inhabitants' , 'lng': 'longitude', 'lat': 'latitude'}, inplace = True)
df.head()


C:\Users\stefa\anaconda3\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,city,number_inhabitants,latitude,longitude
0,Amsterdam,862965,52.350000,4.916667
1,Rotterdam,644618,51.916667,4.500000
2,Den Haag,537833,52.083333,4.300000
3,Utrecht,352866,52.093813,5.119095
4,Eindhoven,231642,51.450000,5.466667


In [16]:
df.shape

(50, 4)

### Plot 50 cities on map

In [17]:
map_netherlands = folium.Map(location=[52.093813,5.119095],zoom_start=7)

for lat,lng,city in zip(df['latitude'],df['longitude'],df['city']):
    label = '{}'.format(city)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_netherlands)
map_netherlands

## Search Foursquare for the selected dutch cities

#### Set API settings

In [18]:
CLIENT_ID = 'xxx' # your Foursquare ID
CLIENT_SECRET = 'xxx' # your Foursquare Secret
VERSION = '20200716' # Foursquare API version
LIMIT = 500

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: EFAEKUHQVI24C0ZWMQUXR1ZTKHDNZPDVYQPDIVVPQZENZN5C
CLIENT_SECRET:VX5OXOVEKAVFNONMTN34R3YCHEGXDM4GMV22JRMHKJGJVEZ4


#### create function to search foursquare for eacht city

In [19]:
import urllib
def getNearbyVenues(names, latitudes, longitudes, radius=5000, categoryIds=''):
    try:
        venues_list=[]
        for name, lat, lng in zip(names, latitudes, longitudes):
            #print(name)

            # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)

            if (categoryIds != ''):
                url = url + '&categoryId={}'
                url = url.format(categoryIds)

            # make the GET request
            response = requests.get(url).json()
            results = response["response"]['venues']

            # return only relevant information for each nearby venue
            for v in results:
                success = False
                try:
                    category = v['categories'][0]['name']
                    success = True
                except:
                    pass

                if success:
                    venues_list.append([(
                        name, 
                        lat, 
                        lng, 
                        v['name'], 
                        v['location']['lat'], 
                        v['location']['lng'],
                        v['categories'][0]['name']
                    )])

        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['City', 
                  'City Latitude', 
                  'City Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude',  
                  'Venue Category']
    
    except:
        print(url)
        print(response)
        print(results)
        print(nearby_venues)

    return(nearby_venues)

#### Call the function

In [20]:
dutch_venues = getNearbyVenues(names=df['city'],
                                latitudes=df['latitude'],
                                longitudes=df['longitude']
                                  )

#### look at content in dataframe

In [21]:
dutch_venues.head()

,City,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Amsterdam,52.35,4.916667,Sportcity,52.349013,4.916926,Gym
1,Amsterdam,52.35,4.916667,Station Amsterdam Amstel,52.346591,4.917637,Train Station
2,Amsterdam,52.35,4.916667,Berkhoff De,52.349658,4.916524,High School
3,Amsterdam,52.35,4.916667,Thuis.,52.354495,4.914572,Residential Building (Apartment / Condo)
4,Amsterdam,52.35,4.916667,Café-Restaurant Dauphine,52.348491,4.917229,French Restaurant


In [22]:
dutch_venues.shape

(4800, 7)

#### See number of vebues for each city

In [23]:
dutch_venues.groupby('City').count()

,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
City,,,,,,
Alkmaar,114,114,114,114,114,114
Almere,90,90,90,90,90,90
Alphen aan den Rijn,101,101,101,101,101,101
Amersfoort,116,116,116,116,116,116
Amstelveen,64,64,64,64,64,64
Amsterdam,109,109,109,109,109,109
Apeldoorn,112,112,112,112,112,112
Arnhem,97,97,97,97,97,97
Breda,66,66,66,66,66,66


#### see number of unique categories

In [24]:
print('There are {} uniques categories.'.format(len(dutch_venues['Venue Category'].unique())))

There are 422 uniques categories.


## Analyze each city

In [25]:
# one hot encoding
dutch_onehot = pd.get_dummies(dutch_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
dutch_onehot['City'] =dutch_venues['City'] 

# move neighborhood column to the first column
fixed_columns = [dutch_onehot.columns[-1]] + list(dutch_onehot.columns[:-1])
dutch_onehot = dutch_onehot[fixed_columns]

dutch_onehot.head()

,City,ATM,Adult Boutique,Advertising Agency,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,Airport Terminal,Alternative Healer,...,Waterfront,Well,Whisky Bar,Wine Bar,Wine Shop,Winery,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,Amsterdam,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Amsterdam,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Amsterdam,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Amsterdam,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Amsterdam,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
dutch_onehot.shape

(4800, 423)

### Group rows by city and by taking the mean of the frequency of occurrence of each category


In [27]:
dutch_grouped = dutch_onehot.groupby('City').mean().reset_index()
dutch_grouped

,City,ATM,Adult Boutique,Advertising Agency,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,Airport Terminal,Alternative Healer,...,Waterfront,Well,Whisky Bar,Wine Bar,Wine Shop,Winery,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,Alkmaar,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Almere,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.011111,0.000000,0.000000,0.000000,0.000000,0.000000,0.011111,0.000000,0.000000
2,Alphen aan den Rijn,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.009901,0.000000,0.000000
3,Amersfoort,0.000000,0.000000,0.000000,0.000000,0.008621,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.034483,0.000000,0.000000,0.043103,0.000000,0.000000,0.000000
4,Amstelveen,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.031250,0.000000,0.000000,0.015625,0.000000,0.000000
5,Amsterdam,0.000000,0.000000,0.009174,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.009174,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Apeldoorn,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Arnhem,0.000000,0.000000,0.010309,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Breda,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.015152,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.015152,0.000000,0.000000,0.000000
9,Delft,0.000000,0.000000,0.000000,0.000000,0.000000,0.008696,0.000000,0.000000,0.000000,...,0.008696,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [28]:
dutch_grouped.shape

(49, 423)

In [29]:
num_top_venues = 10

for hood in dutch_grouped['City']:
    print("----"+hood+"----")
    temp = dutch_grouped[dutch_grouped['City'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Alkmaar----
                                      venue  freq
0                    Furniture / Home Store  0.17
1                                    Office  0.10
2  Residential Building (Apartment / Condo)  0.04
3               Professional & Other Places  0.04
4                             Design Studio  0.04
5                                   Parking  0.03
6                            Hardware Store  0.03
7                        Salon / Barbershop  0.03
8                         Electronics Store  0.03
9                               Gas Station  0.02


----Almere----
                  venue  freq
0                Office  0.11
1                Bridge  0.07
2              Building  0.06
3     Elementary School  0.03
4  Other Great Outdoors  0.03
5  Mental Health Office  0.02
6              Bus Stop  0.02
7                School  0.02
8    Salon / Barbershop  0.02
9        Medical Center  0.02


----Alphen aan den Rijn----
                          venue  freq
0                  

9            Bus Stop  0.02


----Hilversum----
                    venue  freq
0                  Office  0.19
1  Furniture / Home Store  0.07
2                Building  0.05
3     Arts & Crafts Store  0.05
4                 Dog Run  0.05
5             Flea Market  0.02
6             Snack Place  0.02
7              Restaurant  0.02
8         Doctor's Office  0.02
9                 Factory  0.02


----Hoeksche Waard----
                  venue  freq
0                Office  0.11
1              Building  0.11
2       Automotive Shop  0.10
3              Bus Stop  0.05
4      Dentist's Office  0.04
5                School  0.04
6               Library  0.04
7     College Classroom  0.03
8              Bus Line  0.03
9  Gym / Fitness Center  0.03


----Leeuwarden----
                                      venue  freq
0                                    Office  0.10
1                          Dentist's Office  0.06
2                            Medical Center  0.05
3  Residential Building 

                         venue  freq
0                       Office  0.16
1            College Classroom  0.14
2                     Building  0.08
3                      Parking  0.05
4                     Platform  0.04
5  Professional & Other Places  0.03
6                Train Station  0.03
7                 Tech Startup  0.03
8                     Bus Line  0.03
9              Coworking Space  0.03




In [30]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [31]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['City']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
city_venues_sorted = pd.DataFrame(columns=columns)
city_venues_sorted['City'] = dutch_grouped['City']

for ind in np.arange(dutch_grouped.shape[0]):
    city_venues_sorted.iloc[ind, 1:] = return_most_common_venues(dutch_grouped.iloc[ind, :], num_top_venues)

city_venues_sorted.head()

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Alkmaar,Furniture / Home Store,Office,Design Studio,Professional & Other Places,Residential Building (Apartment / Condo),Hardware Store,Parking,Electronics Store,Salon / Barbershop,Mattress Store
1,Almere,Office,Bridge,Building,Elementary School,Other Great Outdoors,Garden,School,Mental Health Office,Medical Center,Bus Stop
2,Alphen aan den Rijn,Office,Medical Center,Bus Stop,Greek Restaurant,Church,Building,Salon / Barbershop,Gym / Fitness Center,Gym,Playground
3,Amersfoort,Bar,Restaurant,Women's Store,Café,Other Nightlife,Shoe Store,Wine Bar,Coffee Shop,Clothing Store,Gay Bar
4,Amstelveen,Dentist's Office,Building,Office,Bakery,Drugstore,Salon / Barbershop,Martial Arts Dojo,Supermarket,Wine Shop,Miscellaneous Shop


## Clustering Cities

In [43]:
# set number of clusters
kclusters = 5

dutch_grouped_clustering = dutch_grouped.drop('City', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dutch_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 2, 1, 1, 1, 2, 2, 1, 1, 2])

In [44]:
city_venues_sorted.head()

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Alkmaar,Furniture / Home Store,Office,Design Studio,Professional & Other Places,Residential Building (Apartment / Condo),Hardware Store,Parking,Electronics Store,Salon / Barbershop,Mattress Store
1,Almere,Office,Bridge,Building,Elementary School,Other Great Outdoors,Garden,School,Mental Health Office,Medical Center,Bus Stop
2,Alphen aan den Rijn,Office,Medical Center,Bus Stop,Greek Restaurant,Church,Building,Salon / Barbershop,Gym / Fitness Center,Gym,Playground
3,Amersfoort,Bar,Restaurant,Women's Store,Café,Other Nightlife,Shoe Store,Wine Bar,Coffee Shop,Clothing Store,Gay Bar
4,Amstelveen,Dentist's Office,Building,Office,Bakery,Drugstore,Salon / Barbershop,Martial Arts Dojo,Supermarket,Wine Shop,Miscellaneous Shop


In [45]:
kmeans.labels_

array([4, 2, 1, 1, 1, 2, 2, 1, 1, 2, 1, 2, 4, 1, 2, 1, 1, 3, 1, 1, 1, 0,
       3, 1, 1, 2, 0, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 4, 2, 0, 1, 1, 0, 2,
       2, 0, 1, 1, 3])

In [46]:
# add clustering labels
city_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_) #Comment beceause it cannot run twice

dutch_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
dutch_merged = dutch_merged.join(city_venues_sorted.set_index('City'), on='city')

dutch_merged.head() # check the last columns!

,city,number_inhabitants,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Amsterdam,862965,52.350000,4.916667,2,Office,Building,Residential Building (Apartment / Condo),Coworking Space,Assisted Living,Bar,Non-Profit,Tech Startup,Playground,Meeting Room
1,Rotterdam,644618,51.916667,4.500000,2,Office,Residential Building (Apartment / Condo),Building,Bar,Bus Stop,Coworking Space,Restaurant,Business Service,Event Space,Housing Development
2,Den Haag,537833,52.083333,4.300000,2,Office,Embassy / Consulate,Art Gallery,Salon / Barbershop,Dentist's Office,Restaurant,Residential Building (Apartment / Condo),Miscellaneous Shop,Plaza,Coworking Space
3,Utrecht,352866,52.093813,5.119095,2,Office,Salon / Barbershop,Snack Place,Clothing Store,Bar,General Entertainment,Tech Startup,Fast Food Restaurant,Shoe Store,Cocktail Bar
4,Eindhoven,231642,51.450000,5.466667,1,Office,Residential Building (Apartment / Condo),General College & University,Furniture / Home Store,Housing Development,Chinese Restaurant,Turkish Restaurant,Music Venue,Bar,Fast Food Restaurant


In [47]:
dutch_merged

,city,number_inhabitants,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Amsterdam,862965,52.350000,4.916667,2,Office,Building,Residential Building (Apartment / Condo),Coworking Space,Assisted Living,Bar,Non-Profit,Tech Startup,Playground,Meeting Room
1,Rotterdam,644618,51.916667,4.500000,2,Office,Residential Building (Apartment / Condo),Building,Bar,Bus Stop,Coworking Space,Restaurant,Business Service,Event Space,Housing Development
2,Den Haag,537833,52.083333,4.300000,2,Office,Embassy / Consulate,Art Gallery,Salon / Barbershop,Dentist's Office,Restaurant,Residential Building (Apartment / Condo),Miscellaneous Shop,Plaza,Coworking Space
3,Utrecht,352866,52.093813,5.119095,2,Office,Salon / Barbershop,Snack Place,Clothing Store,Bar,General Entertainment,Tech Startup,Fast Food Restaurant,Shoe Store,Cocktail Bar
4,Eindhoven,231642,51.450000,5.466667,1,Office,Residential Building (Apartment / Condo),General College & University,Furniture / Home Store,Housing Development,Chinese Restaurant,Turkish Restaurant,Music Venue,Bar,Fast Food Restaurant
5,Groningen,231299,53.216667,6.550000,1,Building,Office,Medical Center,Dentist's Office,Pizza Place,Tech Startup,Professional & Other Places,Residential Building (Apartment / Condo),Bar,Bus Stop
6,Tilburg,217259,51.576922,5.106331,0,Office,Building,Factory,Automotive Shop,Coworking Space,School,Transportation Service,Park,Doctor's Office,College Academic Building
7,Almere,207904,52.383922,5.233585,2,Office,Bridge,Building,Elementary School,Other Great Outdoors,Garden,School,Mental Health Office,Medical Center,Bus Stop
8,Breda,183873,51.566667,4.800000,1,Office,Bus Stop,Residential Building (Apartment / Condo),Gas Station,Professional & Other Places,Cosmetics Shop,Bakery,Parking,Playground,Supermarket
9,Nijmegen,176731,51.833333,5.866667,1,Office,School,Medical Center,Snack Place,Residential Building (Apartment / Condo),Playground,Dentist's Office,High School,Tech Startup,Building


### Visualize cliusters on map

In [48]:
# create map
map_clusters = folium.Map(location=[52.093813,5.119095],zoom_start=7) # Utrecht is centre of the Netherlands

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dutch_merged['latitude'], dutch_merged['longitude'], dutch_merged['city'], dutch_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Show detals on cluster 0


In [50]:
dutch_merged.loc[dutch_merged['Cluster Labels'] == 0]

,city,number_inhabitants,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Tilburg,217259,51.576922,5.106331,0,Office,Building,Factory,Automotive Shop,Coworking Space,School,Transportation Service,Park,Doctor's Office,College Academic Building
16,Haarlemmermeer,154238,52.312058,4.693128,0,Office,Building,Automotive Shop,Hardware Store,Factory,Furniture / Home Store,Restaurant,Pizza Place,Playground,Hobby Shop
26,Westland,108603,51.997897,4.220444,0,Building,Office,Flower Shop,Automotive Shop,Convention Center,Dentist's Office,Bus Stop,Professional & Other Places,Bus Line,Distribution Center
39,Hoeksche Waard,86656,51.817051,4.417294,0,Office,Building,Automotive Shop,Bus Stop,Dentist's Office,School,Library,College Classroom,Bus Line,Gym / Fitness Center
45,Schiedam,77999,51.932178,4.392680,0,Office,Building,Factory,Soccer Field,Athletics & Sports,Automotive Shop,Garden,Lounge,Nightclub,Restaurant


#### Show details cluster 1

In [51]:
dutch_merged.loc[dutch_merged['Cluster Labels'] == 1]

,city,number_inhabitants,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Eindhoven,231642,51.450000,5.466667,1,Office,Residential Building (Apartment / Condo),General College & University,Furniture / Home Store,Housing Development,Chinese Restaurant,Turkish Restaurant,Music Venue,Bar,Fast Food Restaurant
5,Groningen,231299,53.216667,6.550000,1,Building,Office,Medical Center,Dentist's Office,Pizza Place,Tech Startup,Professional & Other Places,Residential Building (Apartment / Condo),Bar,Bus Stop
8,Breda,183873,51.566667,4.800000,1,Office,Bus Stop,Residential Building (Apartment / Condo),Gas Station,Professional & Other Places,Cosmetics Shop,Bakery,Parking,Playground,Supermarket
9,Nijmegen,176731,51.833333,5.866667,1,Office,School,Medical Center,Snack Place,Residential Building (Apartment / Condo),Playground,Dentist's Office,High School,Tech Startup,Building
11,Haarlem,161263,52.366667,4.650000,1,Hospital,Office,Medical Center,Building,Residential Building (Apartment / Condo),Bus Stop,Physical Therapist,Pool,Medical Lab,Bar
12,Arnhem,159267,51.979818,5.925636,1,Building,Office,Playground,Bakery,Doctor's Office,Bus Stop,High School,General Entertainment,Snack Place,Garden
14,Amersfoort,156286,52.156022,5.389526,1,Bar,Restaurant,Women's Store,Café,Other Nightlife,Shoe Store,Wine Bar,Coffee Shop,Clothing Store,Gay Bar
15,Zaanstad,155885,52.439266,4.810557,1,City Hall,Building,Platform,Office,Residential Building (Apartment / Condo),Hotel,Salon / Barbershop,Bus Stop,Bus Line,Convenience Store
17,Den Bosch,154205,51.700196,5.306806,1,Office,Bus Stop,Medical Center,Residential Building (Apartment / Condo),Professional & Other Places,Dog Run,Gas Station,Discount Store,Building,Arts & Crafts Store
19,Zoetermeer,124944,52.057102,4.495946,1,Art Gallery,Furniture / Home Store,Middle Eastern Restaurant,Office,Music Venue,Flower Shop,Restaurant,Diner,French Restaurant,Dentist's Office


#### Show details cluster 2

In [52]:
dutch_merged.loc[dutch_merged['Cluster Labels'] == 2]

,city,number_inhabitants,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Amsterdam,862965,52.350000,4.916667,2,Office,Building,Residential Building (Apartment / Condo),Coworking Space,Assisted Living,Bar,Non-Profit,Tech Startup,Playground,Meeting Room
1,Rotterdam,644618,51.916667,4.500000,2,Office,Residential Building (Apartment / Condo),Building,Bar,Bus Stop,Coworking Space,Restaurant,Business Service,Event Space,Housing Development
2,Den Haag,537833,52.083333,4.300000,2,Office,Embassy / Consulate,Art Gallery,Salon / Barbershop,Dentist's Office,Restaurant,Residential Building (Apartment / Condo),Miscellaneous Shop,Plaza,Coworking Space
3,Utrecht,352866,52.093813,5.119095,2,Office,Salon / Barbershop,Snack Place,Clothing Store,Bar,General Entertainment,Tech Startup,Fast Food Restaurant,Shoe Store,Cocktail Bar
7,Almere,207904,52.383922,5.233585,2,Office,Bridge,Building,Elementary School,Other Great Outdoors,Garden,School,Mental Health Office,Medical Center,Bus Stop
10,Apeldoorn,162445,52.216667,5.966667,2,Office,Building,Medical Center,Electronics Store,Bank,Market,General Entertainment,Food Stand,Gas Station,Government Building
24,Ede,115710,52.035253,5.677743,2,Office,Medical Center,Building,Elementary School,Other Great Outdoors,Non-Profit,Chiropractor,Dentist's Office,Housing Development,Professional & Other Places
29,Delft,103163,52.018601,4.378153,2,Office,Building,Other Great Outdoors,Athletics & Sports,Campground,Playground,Dog Run,Medical Center,General Entertainment,Park
30,Venlo,101603,51.370837,6.175101,2,Office,Music Venue,Bus Stop,Park,Gay Bar,Doctor's Office,Bakery,School,Gas Station,College Lab
36,Hilversum,90238,52.231896,5.179877,2,Office,Furniture / Home Store,Building,Arts & Crafts Store,Dog Run,Construction & Landscaping,Gay Bar,Parking,Law School,Restaurant


#### Show details cluster 3

In [53]:
dutch_merged.loc[dutch_merged['Cluster Labels'] == 3]

,city,number_inhabitants,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Enschede,158986,52.219515,6.891235,3,College Classroom,Office,Building,Design Studio,Professional & Other Places,Residential Building (Apartment / Condo),Tattoo Parlor,Bank,Rest Area,Restaurant
18,Zwolle,127497,52.505751,6.085822,3,Office,College Classroom,Building,Parking,Platform,Coworking Space,Professional & Other Places,Bus Line,Tech Startup,Train Station
38,Heerlen,86832,50.882832,5.962319,3,College Classroom,Office,College Auditorium,College Administrative Building,College Academic Building,Student Center,General College & University,College & University,College Cafeteria,Coworking Space


#### Show details cluster 4

In [54]:
dutch_merged.loc[dutch_merged['Cluster Labels'] == 4]

,city,number_inhabitants,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
27,Alkmaar,108558,52.635818,4.755610,4,Furniture / Home Store,Office,Design Studio,Professional & Other Places,Residential Building (Apartment / Condo),Hardware Store,Parking,Electronics Store,Salon / Barbershop,Mattress Store
31,Deventer,99957,52.250000,6.200000,4,Automotive Shop,Furniture / Home Store,Bus Stop,Building,Dentist's Office,Office,Police Station,Miscellaneous Shop,School,Auto Dealership
47,Roosendaal,77032,51.537721,4.492456,4,Office,Factory,Furniture / Home Store,Building,Automotive Shop,Design Studio,Professional & Other Places,Sandwich Place,Tech Startup,Fast Food Restaurant


In [49]:
dutch_merged.to_excel("output.xlsx")  